# Failed Approaches: Why Standard Methods Fail for Sand

This notebook documents the failure modes encountered when applying
standard mixed finite element methods to Richards equation with
Signorini boundary conditions on sand soil. The sharp van
Genuchten--Mualem constitutive laws create extreme stiffness: the
inverse hydraulic conductivity $K^{-1}$ varies over 10 orders of
magnitude across the pressure head range of a typical column.
This causes Newton divergence, Picard stagnation, and front arrest
across all conventional approaches.

The results motivate the operator splitting (Lie--Trotter) scheme,
which avoids assembling $K^{-1}$ in the convective step entirely.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nitsche_signorini import SAND, CLAY
from nitsche_signorini.constitutive import theta_from_h_np


def Se_np(h, soil):
    """Effective saturation (numpy)."""
    m = 1 - 1 / soil.n
    psi = np.maximum(-h, 0)
    return (1 + (soil.alpha * psi) ** soil.n) ** (-m)


def K_np(h, soil):
    """Hydraulic conductivity K(h) [m/s] (numpy)."""
    m = 1 - 1 / soil.n
    Se = np.clip(Se_np(h, soil), 1e-12, 1 - 1e-12)
    t = Se ** (1 / m)
    bracket = 1 - np.clip(1 - t, 1e-12, 1) ** m
    return soil.K_s * Se ** soil.ell * bracket ** 2

## 1. Constitutive Law Degeneracy

The mixed formulation assembles $K^{-1}$ in the velocity mass matrix.
For sand ($\alpha = 2$, $n = 3$), the hydraulic conductivity drops
from $K_s = 10^{-4}$ m/s at saturation to roughly $10^{-14}$ m/s at
$h = -5$ m. The inverse $K^{-1}$ correspondingly ranges from $10^4$
to $10^{14}$ s/m -- 10 orders of magnitude.

By contrast, clay ($\alpha = 0.2$, $n = 1.5$) has a much gentler
transition. The $K^{-1}$ ratio across the same suction range is
roughly $10^2$, keeping the system well-conditioned.

In [ ]:
h_arr = np.linspace(-5, -1e-3, 1000)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for soil, label, color in [(SAND, 'Sand', 'tab:orange'),
                            (CLAY, 'Clay', 'tab:blue')]:
    K_vals = K_np(h_arr, soil)
    Kinv_vals = 1 / K_vals
    theta_vals = theta_from_h_np(h_arr, soil)

    axes[0].semilogy(h_arr, K_vals, color=color, lw=1.5, label=label)
    axes[1].semilogy(h_arr, Kinv_vals, color=color, lw=1.5, label=label)
    axes[2].plot(h_arr, theta_vals, color=color, lw=1.5, label=label)

axes[0].set(xlabel='h [m]', ylabel='K [m/s]')
axes[0].set_title('(a) Hydraulic conductivity')
axes[1].set(xlabel='h [m]', ylabel='K$^{-1}$ [s/m]')
axes[1].set_title('(b) Inverse conductivity')
axes[2].set(xlabel='h [m]', ylabel='theta [-]')
axes[2].set_title('(c) Water content')
for ax in axes:
    ax.legend()
    ax.grid(True, alpha=0.3)
fig.tight_layout()
plt.show()

## 2. Condition Number vs Column Height

For a column of height $H$ with hydrostatic initial condition
$h = -z$, the driest point is at $z = H$ where $h = -H$. The
$K^{-1}$ condition number of the velocity mass matrix is bounded
below by

$$\kappa(M) \geq \frac{K_s}{K(-H)} = \frac{1}{k_r(-H)}$$

For sand, this grows catastrophically with $H$: $\kappa \approx
10^7$ at $H = 5$ m. For clay, $\kappa$ remains moderate even at
large depths.

In [ ]:
H_arr = np.linspace(0.1, 10, 200)
fig, ax = plt.subplots(figsize=(7, 4))

for soil, label, color in [(SAND, 'Sand', 'tab:orange'),
                            (CLAY, 'Clay', 'tab:blue')]:
    K_at_H = K_np(-H_arr, soil)
    kappa = soil.K_s / K_at_H
    ax.semilogy(H_arr, kappa, color=color, lw=1.5, label=label)

for H_mark in [2, 5]:
    ax.axvline(H_mark, color='grey', ls=':', lw=0.8)
    ax.text(H_mark + 0.1, 1e2, f'H={H_mark}m', fontsize=8, color='grey')

ax.set(xlabel='Column height H [m]', ylabel='Condition number')
ax.set_title('Velocity mass matrix condition number')
ax.legend()
ax.grid(True, alpha=0.3)
fig.tight_layout()
plt.show()

## 3. Monolithic Method on Sand

The monolithic Nitsche--Signorini method (the "base method" of the
paper) solves the full mixed system with Newton's method at each time
step. When the $K^{-1}$ variation is moderate (clay, or sand at small
$H$), Newton converges in 3--5 iterations. As $H$ increases past a
critical threshold, Newton progressively struggles:

- **H = 1 m** ($\kappa \sim 10^3$): converges cleanly, 3--5 iterations.
- **H = 2 m** ($\kappa \sim 10^5$): Newton fails intermittently,
  requiring adaptive time step halving.

Below we run the monolithic method at both heights (sand, $p = 10\,K_s$,
$t_{\mathrm{fin}} = 1$ h) to demonstrate the transition.

In [ ]:
from nitsche_signorini.runners import run_monolithic

p_rain = 10 * SAND.K_s
snap_times = np.arange(600, 3601, 600)

results = {}
for H, maxh, dt in [(1.0, 0.05, 300), (2.0, 0.05, 60)]:
    print()
    print(f'=== H = {H} m ===')
    z, snaps, info = run_monolithic(
        SAND, H, maxh, dt, 3600, p_rain, snap_times, verbose=True)
    nits = info['newton_its']
    nf = info['newton_failures']
    print(f'  Steps: {info["n_steps"]}, Newton failures: {nf}, '
          f'mean={nits.mean():.1f}, max={nits.max()}')
    results[H] = {'z': z, 'snaps': snaps, 'info': info}

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))
cmap = plt.cm.viridis

# (a) theta(z) at H=1m
ax = axes[0]
r = results[1.0]
n_snaps = len(r['snaps'])
for i, (t_s, th_s) in enumerate(r['snaps']):
    c = cmap(i / max(n_snaps - 1, 1))
    label = 'IC' if t_s == 0 else f't={t_s/60:.0f}min'
    ax.plot(th_s, r['z'], color=c, lw=1.2, label=label)
ax.set(xlabel='theta [-]', ylabel='z [m]')
ax.set_title('(a) H=1m: theta(z)')
ax.legend(fontsize=6)

# (b) theta(z) at H=2m
ax = axes[1]
r = results[2.0]
n_snaps = len(r['snaps'])
for i, (t_s, th_s) in enumerate(r['snaps']):
    c = cmap(i / max(n_snaps - 1, 1))
    label = 'IC' if t_s == 0 else f't={t_s/60:.0f}min'
    ax.plot(th_s, r['z'], color=c, lw=1.2, label=label)
ax.set(xlabel='theta [-]', ylabel='z [m]')
ax.set_title('(b) H=2m: theta(z)')
ax.legend(fontsize=6)

# (c) Newton iterations per step
ax = axes[2]
for H, label, color, ms in [(1.0, 'H=1m', 'tab:blue', 3),
                              (2.0, 'H=2m', 'tab:red', 3)]:
    r = results[H]
    nits = r['info']['newton_its']
    nf = r['info']['newton_failures']
    steps = np.arange(1, len(nits) + 1)
    ax.plot(steps, nits, '.-', ms=ms, lw=0.8, color=color,
            label=f'{label} ({nf} failures)')
ax.set(xlabel='Time step', ylabel='Newton iterations')
ax.set_title('(c) Newton convergence')
ax.legend(fontsize=8)

fig.tight_layout()
plt.show()

## 4. Failure Mode Classification

A systematic sweep of column heights ($H = 1$ to $5$ m, sand,
$p/K_s = 10$) reveals four distinct failure regimes, characterised
by the displacement per step diagnostic
$\mathrm{d}h = K_s \cdot \Delta t \,/\, (C \cdot \Delta z)$:

| H [m] | $\kappa(M)$ | dh/step | Status | Mechanism |
|-------|------------|---------|--------|-----------|
| 1.0   | ~$10^3$    | 0.10    | OK     | Clean front descent |
| 1.5   | ~$10^4$    | 0.29    | OK     | Clean front descent |
| 2.0   | ~$10^5$    | 0.67    | BLOWUP | Newton failures, adaptive dt needed |
| 2.5   | ~$10^5$    | 1.30    | VIOLATION | $h > 0$ on seepage face |
| 3.0   | ~$10^6$    | 2.23    | VIOLATION | $h > 0$ on seepage face |
| 5.0   | ~$10^7$    | 10.3    | STAGNATED | Front frozen, no dynamics |

The threshold occurs at dh/step ~ 1, where a single Newton step
demands the front advance by more than one element.

## 5. Alternative Approaches Attempted

Beyond the monolithic method, several alternatives were investigated
for sand. All fail due to the same root cause: $K^{-1}$ blow-up.

### 5.1 Broken RT0 + Nitsche DG Coupling

**Hypothesis:** Breaking inter-element continuity of RT0 would allow
element-wise discontinuities in $h$, enabling shock-like moisture
fronts without the continuity constraint that forces global coupling
through $K^{-1}$.

**Implementation:** Fully discontinuous HDiv with Nitsche DG coupling
on internal facets (consistency + adjoint consistency + penalty terms).
Validated for linear Darcy problems (penalty-insensitive, inf-sup
stable).

**Result: FAILS** for transient Richards on sand. Two failure modes
depending on time step:
- Large dt: jumps to $h = 0$ everywhere in 1--2 steps (spurious
  steady state).
- Small dt: Newton diverges after ~10 steps.

The broken RT0 decouples elements, but without a mechanism to
transfer mass correctly through the $K^{-1}$-weighted flux coupling,
the scheme either absorbs mass unphysically or diverges.

### 5.2 Hybridized RT0 (HDG) + NCP Signorini

**Hypothesis:** HDG with facet Lagrange multipliers provides better
control over Signorini switching via nonlinear complementarity (NCP)
on the multiplier.

**Implementation:** Hybridized RT0 with FacetFESpace for
$\hat{h}$. Static condensation reduces the global system to facet
unknowns only. NCP active-set strategy for Signorini switching.

**Result: FAILS** for sand (clay OK). Three solver variants all
stagnate:
- Standard Newton: diverges after a few time steps.
- L-scheme (Picard with constant $L$): stagnates at residual
  ~0.005.
- L-scheme + Newton polishing: L-scheme stagnates before Newton can
  start.

The facet degrees of freedom inherit the $K^{-1}$ stiffness through
the element-level Schur complement. HDG does not circumvent the
fundamental conditioning problem.

### 5.3 L-scheme + TVB Limiter (Predictor-Corrector)

**Hypothesis:** A TVB (Total Variation Bounded) divergence-preserving
limiter on broken RT0, combined with L-scheme Picard linearisation,
would control oscillations while avoiding Newton entirely.

**Implementation:** Outer loop: L-scheme Picard with constant
$L = \max_h |C(h)|$. Inner loop: TVB limiter on the flux field.
Validated components individually: limiter preserves divergence to
machine precision, L-scheme converges for clay.

**Result: FAILS** for sand at the first time step. The L-scheme
constant $L$ is dominated by the dry-end behaviour, making the
contraction rate $\sim 1 - \varepsilon$. The iteration stagnates
at err ~ 4.6e-3, the time step halves repeatedly, and the solver
aborts.

The limiter itself works correctly; it is the Picard linearisation
of $K^{-1}$ that cannot converge.

### 5.4 Linearised Signorini (Frozen Coefficients)

**Hypothesis:** Freezing $K^{-1}$ and $C$ at the previous time step
avoids Newton entirely. One linear solve per step.

**Result: FAILS** for sand. At $H = 5$ m, the first step produces a
pressure head spike of +37.8 m (physically absurd). The frozen
$K^{-1}$ at $h = -5$ m is so large that the linear system demands an
enormous pressure correction to satisfy the boundary flux. Subsequent
steps stagnate because the correction is clipped, but the front
cannot advance.

With L-scheme damping ($L = 0.52$), the spike is eliminated but the
front displacement per step drops to 0.07 m -- 326x slower than the
physical front speed. The front is effectively frozen.

## 6. Summary

| Method | Clay | Sand ($H \geq 2$ m) | Root cause |
|--------|------|---------------------|------------|
| Monolithic Newton | OK | Newton failures / stagnation | $K^{-1}$ in Jacobian |
| Broken RT0 + Nitsche | OK (linear) | Mass absorption or divergence | $K^{-1}$ in flux coupling |
| HDG + NCP | OK | Picard stagnation | $K^{-1}$ in Schur complement |
| L-scheme + TVB limiter | OK | L-scheme stagnation | $K^{-1}$ in contraction constant |
| Linearised (frozen coeff) | OK | Step-1 spike, front arrest | $K^{-1}$ in stiffness matrix |
| **Operator splitting** | **OK** | **OK** | **$K^{-1}$ bypassed in convection** |

Every standard approach fails because $K^{-1}$ enters the assembled
system -- whether in the velocity mass matrix (monolithic), the Schur
complement (HDG), or the Picard contraction constant (L-scheme). The
operator splitting resolves this by handling the convection step with
Godunov finite volumes, which use $K(\theta)$ directly (no inversion).
The implicit diffusion step sees only the near-identity correction,
where $K^{-1}$ variation is bounded.